In [131]:
import pandas as pd
import numpy as np
import os
import sys
import sklearn
import missingno as msno

In [132]:
#!pip install missingno

In [133]:
raw_df = pd.read_csv('./mimic_cut_3d.csv').set_index('stay_id').drop(columns=['Unnamed: 0'])
raw_df.head(n=10)

,creatinine,fraction_inspired_oxygen,lactate,urine_output,diastolic_blood_pressure,systolic_blood_pressure,gcs,partial_pressure_of_oxygen,heart_rate,temperature,respiratory_rate
stay_id,,,,,,,,,,,
33751102.0,1.150000,0.880000,6.140000,280.000000,57.058333,111.058333,15.0,98.562500,112.104167,36.303175,25.824811
33751102.0,1.150000,0.833333,6.650000,257.142857,56.162963,109.051852,15.0,97.714286,110.759259,36.465278,22.955387
33751102.0,1.150000,0.833333,7.500000,232.500000,55.746667,107.546667,15.0,103.166667,109.683333,36.558025,23.659848
33751102.0,1.166667,0.833333,8.200000,232.500000,55.587879,106.951515,15.0,103.166667,108.712121,36.558025,24.236226
33751102.0,1.166667,0.800000,8.200000,231.666667,55.538889,107.288889,15.0,103.166667,107.486111,36.572222,24.716540
33751102.0,1.150000,0.766667,8.200000,219.000000,54.319444,106.736111,15.0,108.600000,105.569444,36.719444,25.340278
35520609.0,1.575000,0.500000,1.966667,55.181818,76.717172,120.684343,15.0,62.750000,130.594017,38.074074,22.940171
35520609.0,1.633333,0.533333,1.700000,53.700000,76.194444,120.638889,15.0,72.250000,129.388889,37.977778,23.388889
35520609.0,1.633333,0.533333,1.700000,57.200000,76.833333,122.083333,15.0,72.250000,128.416667,37.977778,23.833333


In [134]:
def mask_random_data(data, mask_fraction=0.1):
    """
    Randomly masks a given fraction of the data in a DataFrame.

    :param data: Pandas DataFrame to be masked.
    :param mask_fraction: Fraction of the data to be masked. Default is 0.1 (10%).
    :return: DataFrame with masked data.
    """
    # Flatten the DataFrame to work with individual elements
    flattened_data = data.to_numpy().flatten()

    # Calculate the number of elements to mask
    num_to_mask = int(len(flattened_data) * mask_fraction)

    # Randomly select indices to mask
    mask_indices = np.random.choice(len(flattened_data), num_to_mask, replace=False)

    # Mask the selected indices
    for idx in mask_indices:
        flattened_data[idx] = np.nan

    # Reshape the data back to its original shape and create a new DataFrame
    return pd.DataFrame(flattened_data.reshape(data.shape), columns=data.columns)

In [135]:
mu, std = raw_df.mean(axis=0).values, raw_df.std(axis=0).values
normalized_df = (raw_df - mu)/std
masked_df = mask_random_data(normalized_df)
masked_df.head(n=10)

,creatinine,fraction_inspired_oxygen,lactate,urine_output,diastolic_blood_pressure,systolic_blood_pressure,gcs,partial_pressure_of_oxygen,heart_rate,temperature,respiratory_rate
0,-0.543570,1.732144,0.701248,2.429507,0.108413,0.432360,0.264455,-0.579785,0.955194,-0.728371,NaN
1,-0.543570,1.463702,0.857793,2.149416,0.005184,NaN,0.264455,-0.594241,0.886037,-0.541851,0.362902
2,-0.543570,1.463702,1.118701,1.847442,-0.042812,NaN,0.264455,NaN,0.830711,-0.435134,NaN
3,-0.532450,1.463702,1.333567,1.847442,-0.061119,0.058886,0.264455,-0.501313,0.780770,NaN,0.626581
4,-0.532450,1.271958,1.333567,1.837230,-0.066767,0.089567,0.264455,-0.501313,0.717727,-0.418798,0.725460
5,-0.543570,1.080214,1.333567,1.682013,-0.207359,0.039297,0.264455,-0.408710,NaN,-0.249401,0.853865
6,-0.260024,-0.453738,-0.579760,-0.325415,2.374915,1.307749,0.264455,-1.190158,1.905966,1.309268,0.359770
7,-0.221106,-0.261994,-0.661613,-0.343573,2.314649,1.303615,0.264455,-1.028244,1.843997,NaN,0.452144
8,-0.221106,-0.261994,NaN,-0.300684,2.388307,1.434973,0.264455,-1.028244,1.794004,1.198468,0.543640
9,-0.221106,-0.261994,-0.661613,-0.270049,2.119293,1.230358,0.264455,-1.077955,1.738298,1.245345,0.526484


In [136]:
from sklearn.decomposition import TruncatedSVD
from sklearn.utils.extmath import randomized_svd

def svd_per_patient(masked_df, k=11, max_iter=500, tol=1e-4):
    # Replace NaN values with mean
    imputed_matrix = np.array([]).reshape(0, 11)
    for i in range(len(masked_df) // 6):
        patient_df = masked_df.iloc[i*6:i*6+6]
        updated_patient_matrix = patient_df.to_numpy()

        na_indices = patient_df.isna().to_numpy()
        patient_df = patient_df.fillna(method='ffill').fillna(patient_df.mean())
        patient_matrix = patient_df.to_numpy()

        for _ in range(max_iter):
            # Perform SVD
            #U, S, Vt = np.linalg.svd(patient_matrix, full_matrices=False)

               
            U, S, Vt = randomized_svd(patient_matrix, 
                              n_components=k,
                              n_iter=500,
                              random_state=None,
                              )
            U_k = U[:, :k]
            S_k = np.diag(S[:k])
            Vt_k = Vt[:k, :]
            # Reconstruct matrix with the low-rank approximation
            pred = np.dot(U_k, np.dot(S_k, Vt_k))
            
            updated_patient_matrix[na_indices] = pred[na_indices]
            #print(na_indices)
            #print('---')
            #print(updated_patient_matrix)
            #print('---')
            #print(pred)
            
            # Check for convergence
            if np.linalg.norm(updated_patient_matrix - pred) < tol:
                break
        #print(np.linalg.norm(updated_patient_matrix - y[i:i+6]))
        imputed_matrix = np.concatenate((imputed_matrix, updated_patient_matrix), axis=0)
        
    return imputed_matrix

# Sample data
#imputed_matrix = svd_imputation(masked_df)

In [159]:
masked_df = mask_random_data(normalized_df)
print(masked_df.isna().sum())
na_indices = masked_df.isna().to_numpy()
masked_df.head(n=10)
imputed_matrix = svd_per_patient(masked_df)

creatinine                    326
fraction_inspired_oxygen      311
lactate                       327
urine_output                  325
diastolic_blood_pressure      293
systolic_blood_pressure       303
gcs                           283
partial_pressure_of_oxygen    285
heart_rate                    300
temperature                   308
respiratory_rate              311
dtype: int64


In [162]:
def RMSD_global(y, pred, na_indices):
    results = []
    assert y.shape == pred.shape
    assert y.shape == na_indices.shape
    for i in range(y.shape[1]):
        y_i, pred_i = y[:, i][na_indices[:, i]], pred[:, i][na_indices[:, i]]
        #print(y_i, pred_i)
        var = np.square(y_i - pred_i) 
        rmsd = np.sum(var) / y_i.shape[0]
        results.append(rmsd)
    return np.array(results)

        

In [163]:
y = normalized_df.to_numpy()
rmsd_global = RMSD_global(y, imputed_matrix, na_indices)

In [164]:
print(np.mean(rmsd_global))

0.07656806781891566
